# Get OSM Buildings

In [1]:
import leafmap

m=leafmap.Map(center=[41.88, 12.5597], zoom=13)
m

Map(center=[41.88, 12.5597], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import warnings
import urllib3
import numpy as np
# Optionally, disable Warning and SSL verification
#urllib3.disable_warnings()
#warnings.simplefilter('ignore', FutureWarning)
#ox.config(requests_kwargs={'verify':False})


bbox = tuple(m.get_bbox())
north, south, east, west = bbox[3],bbox[1],bbox[2],bbox[0]

buildings = ox.features_from_bbox(bbox, tags={'building': True,"type":"multipolygon"}).reset_index()

buildings

# Get OSM Railways

In [ ]:
import osmnx as ox
import geopandas as gpd
import glob
import pandas as pd

# Imposta il luogo di interesse (Italia)
regions = [
    "Abruzzo",
    "Basilicata",
    "Calabria",
    "Campania",
    "Emilia-Romagna",
    "Friuli-Venezia Giulia",
    "Lazio",
    "Liguria",
    "Lombardia",
    "Marche",
    "Molise",
    "Piemonte",
    "Puglia",
    "Sardegna",
    "Sicilia",
    "Toscana",
    "Trentino-Alto Adige",
    "Umbria",
    "Valle d'Aosta",
    "Veneto"
]

# Scarica le geometrie dei ponti ferroviari
# Filtriamo per i ponti (bridge) e i tracciati ferroviari (railway)
tags = {"bridge": 'yes', "railway": 'rail'}
for region in regions:
    # Ottieni i dati
    gdf = ox.geometries_from_place(region, tags=tags)

    gdf.reset_index(inplace=True)
    gdf = gdf[gdf['element_type']=='way']

    #gdf = gpd.GeoDataFrame(geometry=gdf.geometry)

    gdf.to_file(f"{region}.geojson", driver="GeoJSON")    



# Percorso ai file GeoJSON
path = './*.geojson'  # Sostituisci con il percorso corretto

# Trova tutti i file GeoJSON nella directory specificata
geojson_files = glob.glob(path)

# Inizializza una lista per i GeoDataFrame
gdf_list = []

# Carica ogni file GeoJSON e aggiungilo alla lista
for file in geojson_files:
    gdf = gpd.read_file(file)
    gdf = gdf[(gdf['railway']=='rail') & (gdf['bridge']=='viaduct')]
    gdf_list.append(gdf)

# Concatenare tutti i GeoDataFrame in uno solo
merged_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

# Salva il GeoDataFrame unificato come GeoJSON
output_file = 'Italy_railways_viaduct.geojson'  # Sostituisci con il nome del file di output
merged_gdf.to_file(output_file, driver='GeoJSON')

print(f"GeoJSON unito salvato come: {output_file}")
